<a href="https://colab.research.google.com/github/MossbauerLab/ManometerNNAn/blob/master/sm2201_manometer_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Получение списка изображений с гугл диска, изображения представляют собой классификацию положения стрелки манометра
шкала (0-1,6 атм.)
Изображения рассортированы по директориям с положением стрелки манометра около соответсвующего значения шкалы:

*   0,1 атм
*   0,2 атм

...

*   1,6 атм

Для поиска варианта при котором наиболее оптимальный результат оценки работы нейросети изображения имеют разную степень свертки (оригинал, 128х128 пикселей, ...)
Оригинал имеет имя типа {Value}.{Number}
где {Value} - величина давления
       {Number} - какое-то число (номер изображения)
       
Сжатые изображения имеют имя типа  {Value}.{Number}_{WxH}
где {WxH} - величина изображения в пикселях ширина х высота

ФОРМАТ ИЗОБРАЖЕНИЙ - .jpg


Монтирование гугл диска

In [0]:
from google.colab import drive
drive.mount(u'/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import VGG16


# list root dir
# for item in os.listdir(u'/content/gdrive/My Drive'):
#    print(u'sub folder item: ' + item)

manometer_nn_data_path = os.path.join(u'/content/gdrive', 'My Drive/MessbauerSoft/NeuralNetworks_Data/Manometer')
print (u'path to root folder with manometer neural network data is:' + os.path.abspath(manometer_nn_data_path))

# manometer train data = test (it is not good, but we are going to use this case)
train_data_path = os.path.join(manometer_nn_data_path, u'train')
test_data_path = os.path.join(manometer_nn_data_path, u'test')
validate_data_path = os.path.join(manometer_nn_data_path, u'validate')

# list sub folder (classification items)
'''
for item in os.listdir(train_data_path):
    print(u'sub folder item: ' + item)
'''
img_width, img_height = 128, 128

input_shape = (img_width, img_height, 3)

epochs = 30

batch_size = 6

# todo: umv: count images
# nb_train_samples = 35

vgg16_net = VGG16(include_top=False, input_shape=input_shape, pooling='max')
vgg16_net.summary()

model = Sequential()
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(33))
model.add(Activation('softmax'))
model.summary()

# optimizer SGD - Stochastic gradient descent optimizer, has a lot of params
# see optimizers here: https://keras.io/optimizers/
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(train_data_path,
                                              target_size=(img_width, img_height),
                                              batch_size=batch_size,
                                              class_mode='categorical')

# i don't know would we use validation or not ...
validation_generator = datagen.flow_from_directory(validate_data_path, 
                                                   target_size=(img_width, img_height),
                                                   batch_size=batch_size,
                                                   class_mode='categorical')

test_generator = datagen.flow_from_directory(test_data_path,
                                             target_size=(img_width, img_height),
                                             batch_size=batch_size,
                                             class_mode='categorical')

model.fit_generator(train_generator, 
                    #steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=None
                    #validation_steps=nb_validation_samples // batch_size
                    ) 




path to root folder with manometer neural network data is:/content/gdrive/My Drive/MessbauerSoft/NeuralNetworks_Data/Manometer
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_______________